In [2]:
%load_ext autoreload
%load_ext autotime

time: 334 µs (started: 2023-02-12 10:05:52 -08:00)


In [3]:
import sys
sys.path.append("../")
import warnings
warnings.filterwarnings('ignore')
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

time: 1.65 s (started: 2023-02-12 10:05:52 -08:00)


In [4]:
import dask.dataframe as dd
import numpy as np
import pandas as pd


time: 1.09 s (started: 2023-02-12 10:05:54 -08:00)


In [5]:
%autoreload 2

from wisio.recorder import RecorderAnalyzer

log_dir = "/p/vast1/iopp/recorder_app_logs/cm1/nodes-32/workflow-4/_parquet"
    # log_dir = "/p/gpfs1/iopp/recorder_app_logs/hacc/nodes-32/workflow-0/_parquet"
    # log_dir = "/p/gpfs1/iopp/recorder_app_logs/montage/nodes-32/_parquet"
    # log_dir = "/p/gpfs1/iopp/recorder_app_logs/genome_pegasus/nodes-32/_parquet"
    # log_dir = "/p/gpfs1/iopp/recorder_app_logs/lbann-jag/nodes-32/_parquet"

recorder_analyzer = RecorderAnalyzer(
    cluster_manager_args=dict(
        force_local=True,
        cluster_settings=dict(
            cores=4,
            dashboard_port=3646,
            local_directory="/var/tmp/dask-recorder",
            log_file="%J.log",
            worker_queue="pdebug"
        )
    ),
    working_dir='.recorder',
    debug=True
) 

recorder_analyzer

time: 2.44 s (started: 2023-02-12 10:05:55 -08:00)


In [6]:
recorder_analyzer.cluster_manager.clients

[<Client: 'tcp://192.168.132.138:40766' processes=4 threads=36>]

time: 2.77 ms (started: 2023-02-12 10:05:58 -08:00)


In [141]:
%autoreload 2

res = recorder_analyzer.analyze_parquet(log_dir=log_dir)

('trange',) 6045
('file_id',) 6828
('proc_id',) 6518
('trange', 'file_id') 3998
('trange', 'proc_id') 2307
('file_id', 'trange') 4022
('file_id', 'proc_id') 4332
('proc_id', 'trange') 2307
('proc_id', 'file_id') 4266
('trange', 'file_id', 'proc_id') 1506
('trange', 'proc_id', 'file_id') 1442
('file_id', 'trange', 'proc_id') 1530
('file_id', 'proc_id', 'trange') 1530
('proc_id', 'trange', 'file_id') 1442
('proc_id', 'file_id', 'trange') 1466
time: 10.3 s (started: 2023-02-12 14:07:34 -08:00)


In [142]:
views = res

time: 8 ms (started: 2023-02-12 14:07:47 -08:00)


In [194]:
from wisio._recorder.analysis import compute_unique_filenames, compute_unique_processes
%autoreload 2

file_id_views = []
proc_id_views = []

for view_key, view in views.items():
    if 'file_id' in view_key[-1]:
        file_id_views.append(view['file_id'].unique())
    elif 'proc_id' in view_key[-1]:
        proc_id_views.append(view['proc_id'].unique())

unique_file_ids = list(set(dd.concat(file_id_views).compute()))
unique_proc_ids = list(set(dd.concat(proc_id_views).compute()))

unique_filenames = compute_unique_filenames(log_dir=log_dir, unique_file_ids=unique_file_ids)
unique_processes = compute_unique_processes(log_dir=log_dir, unique_proc_ids=unique_proc_ids)


time: 2.33 s (started: 2023-02-12 17:26:00 -08:00)


In [200]:
from wisio.rules import Rule
from wisio._recorder.rules import RecorderRuleEngine
%autoreload 2


# Create rule engine
rule_engine = RecorderRuleEngine(
    rules=list(Rule),
    views=views,
    unique_filenames=unique_filenames,
    unique_processes=unique_processes
)

# Process rules
bottlenecks = rule_engine.process_rules()


time: 10.1 s (started: 2023-02-12 19:27:28 -08:00)


In [201]:
bottlenecks[('trange',)].compute()


,trange,file_id,proc_id,reason
rule,,,,
TIME_RANGE_EXCESSIVE_IO_TIME,[5],None,None,The application spends 66.78s (68.77%) of its ...
FILE_EXCESSIVE_IO_TIME,None,"[4531734247156679037, 4531734245058167199]",None,The application spends 68.77% of its I/O time ...
PROCESS_EXCESSIVE_IO_TIME,None,None,"[6860131432629070484, 6860131432615963284, 686...",The application spends 68.77% of its I/O time ...


time: 963 ms (started: 2023-02-12 19:27:40 -08:00)


In [202]:
list(bottlenecks[('trange',)]['reason'].compute())


['The application spends 66.78s (68.77%) of its I/O time between 4-5s',
 'The application spends 68.77% of its I/O time 1) 64.80s (66.74%) on file /p/gpfs1/iopp/temp/cm1r20.3.2846827/config_files/les_ConvBoundLayer/cm1out_000001_s.dat (very high) 2) 1.97s (2.03%) on file /p/gpfs1/iopp/temp/cm1r20.3.2846827/config_files/les_ConvBoundLayer/cm1out_stats.ctl (very high)',
 'The application spends 68.77% of its I/O time 1) 2.19s (2.26%) on rank 600 (very high) 2) 2.19s (2.26%) on rank 400 (very high) 3) 2.17s (2.23%) on rank 80 (very high) 4) 2.17s (2.23%) on rank 360 (very high) 5) 2.16s (2.23%) on rank 560 (very high) 6) 2.16s (2.23%) on rank 960 (very high) 7) 2.16s (2.23%) on rank 520 (very high) 8) 2.16s (2.23%) on rank 720 (very high) 9) 2.16s (2.23%) on rank 880 (very high) 10) 2.15s (2.22%) on rank 1000 (very high) 11) 2.15s (2.22%) on rank 200 (very high) 12) 2.15s (2.22%) on rank 1240 (very high) 13) 2.14s (2.20%) on rank 280 (very high) 14) 2.13s (2.19%) on rank 120 (very high) 1

time: 1.1 s (started: 2023-02-12 19:27:49 -08:00)


In [150]:
bottlenecks[('trange',)][Rule.FILE_EXCESSIVE_IO_TIME].compute()

,file_id,proc_id,trange,reason
rule,,,,
FILE_EXCESSIVE_IO_TIME,"[4531734247156679037, 4531734247515089012, 453...",None,None,The application spends 98.77% of its I/O time ...


time: 204 ms (started: 2023-02-12 14:10:55 -08:00)


In [152]:
dd.concat([bottlenecks[('trange',)][Rule.TIME_RANGE_EXCESSIVE_IO_TIME], bottlenecks[('trange',)][Rule.FILE_EXCESSIVE_IO_TIME]]).compute()

,trange,file_id,proc_id,reason
rule,,,,
TIME_RANGE_EXCESSIVE_IO_TIME,[5],None,None,The application spends 66.78s (68.77%) of its ...
FILE_EXCESSIVE_IO_TIME,None,"[4531734247156679037, 4531734247515089012, 453...",None,The application spends 98.77% of its I/O time ...


time: 395 ms (started: 2023-02-12 14:12:40 -08:00)


In [144]:
list(bottlenecks[('trange',)][1]['reason'].compute())

['The application spends 98.77% of its I/O time within 1) 64.81s (66.75%) between 4531734247156679037 (very high) 2) 5.81s (5.99%) between 4531734247515089012 (very low) 3) 4.22s (4.34%) between 4531734246187791667 (low) 4) 4.12s (4.24%) between 4531734247569105940 (low) 5) 3.80s (3.91%) between 4531734247572069952 (low) 6) 3.66s (3.77%) between 4531734245572410724 (low) 7) 2.31s (2.37%) between 4531734247394096880 (low) 8) 2.08s (2.15%) between 4531734247092377932 (low) 9) 1.99s (2.05%) between 4531734245058167199 (very high) 10) 1.09s (1.12%) between 4531734247135877453 (low) 11) 0.36s (0.37%) between 4531734244977734007 (very low) 12) 0.33s (0.33%) between 4531734244909082846 (very low) 13) 0.20s (0.21%) between 4531734245629303532 (low) 14) 0.20s (0.20%) between 4531734245256545024 (low) 15) 0.19s (0.19%) between 4531734247071939659 (low) 16) 0.08s (0.08%) between 4531734244992817467 (very low) 17) 0.04s (0.04%) between 4531734245212906884 (low) 18) 0.04s (0.04%) between 4531734247

time: 203 ms (started: 2023-02-12 14:07:52 -08:00)


In [198]:
trange_view = views[('trange',)].copy()
trange_view.columns = ['_'.join(tup).rstrip('_') for tup in trange_view.columns.values]

trange_view_top = trange_view \
    .groupby(['trange']) \
    .agg({'duration_sum': sum, 'duration_pero': sum, 'duration_score': 'first', 'duration_cut': 'first'}) \
    .sort_values('duration_sum', ascending=False) \
    .reset_index()

trange_view_top['rule'] = 1

rule_column = 'trange'

trange_view_rule = trange_view_top.groupby('rule').agg({
    rule_column: list,
    'duration_sum': list,
    'duration_pero': list
})


def set_reason(row):
    print(row.values)
    print(type(row))
    if not row.empty:
        print(row)
    return f"{row['duration_pero'][0]:.2f}"

trange_view_rule['duration_tot'] = trange_view['duration_sum'].sum()
trange_view_rule['reason'] = trange_view_rule.apply(set_reason, axis=1, meta=str)
trange_view_rule.compute()


[list([5, 450, 668, 344, 233, 128, 561, 129, 345, 449, 667, 634, 3, 7, 185, 150, 361, 562, 613, 234, 408, 60, 6, 199, 454, 669, 556])
 list([66.77576446533203, 5.333155155181885, 4.466335773468018, 4.180270195007324, 3.813164710998535, 3.7199175357818604, 2.4576210975646973, 2.2699739933013916, 1.1589834690093994, 0.5583770871162415, 0.3500378131866455, 0.32810160517692566, 0.10001970082521439, 0.056010399013757706, 0.03346849977970123, 0.031905800104141235, 0.030776802450418472, 0.029265599325299263, 0.028370600193738937, 0.027478599920868874, 0.027275599539279938, 0.025101499632000923, 0.02481280080974102, 0.023606998845934868, 0.022143499925732613, 0.018400199711322784, 0.01044359989464283])
 list([0.6877409815788269, 0.05492755398154259, 0.045999955385923386, 0.04305369034409523, 0.03927277401089668, 0.0383123978972435, 0.025311678647994995, 0.023379052057862282, 0.011936672963202, 0.005750870797783136, 0.003605130361393094, 0.0033792038448154926, 0.0010301289148628712, 0.000576865

,trange,duration_sum,duration_pero,duration_tot,reason
rule,,,,,
1,"[5, 450, 668, 344, 233, 128, 561, 129, 345, 44...","[66.77576446533203, 5.333155155181885, 4.46633...","[0.6877409815788269, 0.05492755398154259, 0.04...",95.900772,0.69


time: 264 ms (started: 2023-02-12 19:16:49 -08:00)


In [ ]:
dd.DataFrame()

In [97]:
views[('trange',)].compute().groupby([('duration', 'score')]).count()

trange file_id proc_id io_cat acc_pat       duration index  size             duration      
                                                    min   max      sum count   min   max   sum     pero  perr
(duration, score)                                                                                            
1                   4739    4739    4739   4739    4739  4739     4739  4739  4739  4739  4739     4739  4739
2                   1274    1274    1274   1274    1274  1274     1274  1274  1274  1274  1274     1274  1274
5                     32      32      32     32      32    32       32    32    32    32    32       32    32

time: 140 ms (started: 2023-02-11 20:30:35 -08:00)


In [23]:
from wisio._recorder.analysis import DELTA_BINS

np.choose(np.subtract(np.digitize([0.68], bins=DELTA_BINS, right=True), 1), DELTA_BINS)


array([0.5])

time: 3.87 ms (started: 2023-02-12 10:19:24 -08:00)


In [134]:
views[('trange',)].compute()['duration', 'score'].value_counts(bins=np.arange(8))


(-0.001, 1.0]    4739
(1.0, 2.0]       1274
(4.0, 5.0]         32
(2.0, 3.0]          0
(3.0, 4.0]          0
(5.0, 6.0]          0
(6.0, 7.0]          0
Name: (duration, score), dtype: int64

time: 137 ms (started: 2023-02-11 20:55:29 -08:00)


In [47]:
views[('trange',)]

Dask DataFrame Structure:
              trange file_id proc_id io_cat acc_pat       duration  index   size               duration                  
                                                min   max      sum  count    min    max    sum    score     pero     perr
npartitions=1                                                                                                            
               int64   int64   int64  int64    int8  int8  float32  int64  int64  int64  int64    int64  float32  float32
                 ...     ...     ...    ...     ...   ...      ...    ...    ...    ...    ...      ...      ...      ...
Dask Name: set_metric_percentage, 17 tasks

time: 37.5 ms (started: 2023-02-11 18:01:36 -08:00)


In [24]:
copy_trange_view = views[('trange',)].copy()
copy_trange_view.columns = ['_'.join(tup).rstrip('_') for tup in copy_trange_view.columns.values]
trange_view_group_top = copy_trange_view.groupby(['trange']).agg(
    {'duration_sum': sum, 'duration_pero': sum, 'duration_score': 'first'}).sort_values('duration_pero', ascending=False).head()
trange_view_group_top


,duration_sum,duration_pero,duration_score
trange,,,
5,66.775764,0.687741,5
450,5.333155,0.054928,2
668,4.466336,0.046000,2
344,4.180270,0.043054,2
233,3.813165,0.039273,2


time: 175 ms (started: 2023-02-11 17:45:34 -08:00)


In [27]:
trange_view_group_top['duration_pero'].sum()

0.8709949

time: 3.54 ms (started: 2023-02-11 17:46:01 -08:00)


In [37]:
for i, (index, data) in enumerate(trange_view_group_top.iterrows()):
    print(i, index, data['duration_sum'], data['duration_pero'], data['duration_score'])

0 5 66.77576446533203 0.6877409815788269 5.0
1 450 5.333155155181885 0.05492755398154259 2.0
2 668 4.466335773468018 0.045999955385923386 2.0
3 344 4.180270195007324 0.04305369034409523 2.0
4 233 3.813164710998535 0.03927277401089668 2.0
time: 3.8 ms (started: 2023-02-11 17:54:49 -08:00)


In [12]:
trange_view = views[('trange',)].compute()
trange_view_top = trange_view[trange_view['duration', 'score'] == 5]
trange_view_top

trange              file_id              proc_id io_cat acc_pat      \
                                                                 min max   
3834      5  4531734245058167199  6860131432589748884      3       0   0   
3835      5  4531734247156679037  6860131432592370324      3       0   0   
3836      5  4531734247156679037  6860131432594991764      3       0   0   
3837      5  4531734247156679037  6860131432597613204      3       0   0   
3838      5  4531734247156679037  6860131432600234644      3       0   0   
3839      5  4531734247156679037  6860131432602856084      3       0   0   
3840      5  4531734247156679037  6860131432605477524      3       0   0   
3841      5  4531734247156679037  6860131432608098964      3       0   0   
3842      5  4531734247156679037  6860131432610720404      3       0   0   
3843      5  4531734247156679037  6860131432613341844      3       0   0   
3844      5  4531734247156679037  6860131432615963284      3       0   0   
3845      5  4531734247156679037  6860131432618584724      3       0   0   
3846      5  4531734247156679037  6860131432621206164      3       0   0   
3847      5  4531734247156679037  6860131432623827604      3       0   0   
3848      5  4531734247156679037  6860131432626449044      3       0   0   
3849      5  4531734247156679037  6860131432629070484      3       0   0   
3850      5  4531734247156679037  6860131432631691924      3       0   0   
3851      5  4531734247156679037  6860131432634313364      3       0   0   
3852      5  4531734247156679037  6860131432636934804      3       0   0   
3853      5  4531734247156679037  6860131432639556244      3       0   0   
3854      5  4531734247156679037  6860131432642177684      3       0   0   
3855      5  4531734247156679037  6860131432644799124      3       0   0   
3856      5  4531734247156679037  6860131432647420564      3       0   0   
3857      5  4531734247156679037  6860131432650042004      3       0   0   
3858      5  4531734247156679037  6860131432652663444      3       0   0   
3859      5  4531734247156679037  6860131432655284884      3       0   0   
3860      5  4531734247156679037  6860131432657906324      3       0   0   
3861      5  4531734247156679037  6860131432660527764      3       0   0   
3862      5  4531734247156679037  6860131432663149204      3       0   0   
3863      5  4531734247156679037  6860131432665770644      3       0   0   
3864      5  4531734247156679037  6860131432668392084      3       0   0   
3865      5  4531734247156679037  6860131432671013524      3       0   0   

      duration index size         duration                      
           sum count  min max sum    score      pero      perr  
3834  1.970968     1    0   0   0        5  0.020300  0.020552  
3835  1.939239     1    0   0   0        5  0.019973  0.020221  
3836  2.167557     1    0   0   0        5  0.022324  0.022602  
3837  2.129455     1    0   0   0        5  0.021932  0.022205  
3838  2.069059     1    0   0   0        5  0.021310  0.021575  
3839  2.151693     1    0   0   0        5  0.022161  0.022437  
3840  2.054112     1    0   0   0        5  0.021156  0.021419  
3841  2.139112     1    0   0   0        5  0.022031  0.022305  
3842  2.054134     1    0   0   0        5  0.021156  0.021419  
3843  2.165936     1    0   0   0        5  0.022308  0.022585  
3844  2.191390     1    0   0   0        5  0.022570  0.022851  
3845  1.992225     1    0   0   0        5  0.020518  0.020774  
3846  2.114179     1    0   0   0        5  0.021774  0.022045  
3847  2.164664     1    0   0   0        5  0.022294  0.022572  
3848  2.164769     1    0   0   0        5  0.022296  0.022573  
3849  2.191843     1    0   0   0        5  0.022574  0.022855  
3850  2.099467     1    0   0   0        5  0.021623  0.021892  
3851  1.970162     1    0   0   0        5  0.020291  0.020544  
3852  2.164277     1    0   0   0        5  0.022290  0.022568  
3853  1.952886     1    0   0   0        5  0.020113  0

time: 176 ms (started: 2023-02-11 17:02:07 -08:00)


In [26]:
pd.set_option('display.max_colwidth',1000)

trange_view_top.agg({('trange', ''): 'unique', ('file_id', ''): ['unique', 'nunique'], ('proc_id', ''): ['unique', 'nunique']})

,trange,file_id,proc_id
,,,
unique,[5],"[4531734245058167199, 4531734247156679037]","[6860131432589748884, 6860131432592370324, 6860131432594991764, 6860131432597613204, 6860131432600234644, 6860131432602856084, 6860131432605477524, 6860131432608098964, 6860131432610720404, 6860131432613341844, 6860131432615963284, 6860131432618584724, 6860131432621206164, 6860131432623827604, 6860131432626449044, 6860131432629070484, 6860131432631691924, 6860131432634313364, 6860131432636934804, 6860131432639556244, 6860131432642177684, 6860131432644799124, 6860131432647420564, 6860131432650042004, 6860131432652663444, 6860131432655284884, 6860131432657906324, 6860131432660527764, 6860131432663149204, 6860131432665770644, 6860131432668392084, 6860131432671013524]"
nunique,NaN,2,32


time: 21.6 ms (started: 2023-02-07 10:47:36 -08:00)


In [30]:
views[('file_id',)].compute().groupby(['file_id']).agg({('duration', 'sum'): sum, ('duration', 'csp'): 'first', ('duration', 'delta'): 'first', ('duration', 'score'): 'first'}).sort_values(('duration', 'sum'), ascending=False).groupby([('duration', 'score')]).count()

duration          
                       sum csp delta
(duration, score)                   
1                       31  31    31
2                        9   9     9
5                        1   1     1

time: 183 ms (started: 2023-02-07 11:10:17 -08:00)


In [27]:
file_id_view = views[('file_id',)].compute()
file_id_view_top = file_id_view[file_id_view['duration', 'score'] == 5]
file_id_view_top

trange              file_id              proc_id io_cat acc_pat      \
                                                                 min max   
3289      4  4531734247156679037  6860131432592370324      3       0   0   
3290      4  4531734247156679037  6860131432594991764      3       0   0   
3291      4  4531734247156679037  6860131432597613204      3       0   0   
3292      4  4531734247156679037  6860131432600234644      3       0   0   
3293      4  4531734247156679037  6860131432602856084      3       0   0   
...     ...                  ...                  ...    ...     ...  ..   
3412      7  4531734247156679037  6860131432660527764      3       0   0   
3413      7  4531734247156679037  6860131432663149204      3       0   0   
3414      7  4531734247156679037  6860131432665770644      3       0   0   
3415      7  4531734247156679037  6860131432668392084      3       0   0   
3416      7  4531734247156679037  6860131432671013524      3       0   0   

      duration index size         duration                                    \
           sum count  min max sum    score       csp     delta          pero   
3289  0.000003     1    0   0   0        5  0.667527  0.667527  3.192770e-08   
3290  0.000004     1    0   0   0        5  0.667527  0.667527  4.119704e-08   
3291  0.000004     1    0   0   0        5  0.667527  0.667527  4.119704e-08   
3292  0.000003     1    0   0   0        5  0.667527  0.667527  3.192770e-08   
3293  0.000003     1    0   0   0        5  0.667527  0.667527  2.883793e-08   
...        ...   ...  ...  ..  ..      ...       ...       ...           ...   
3412  0.000019     1    0   0   0        5  0.667527  0.667527  1.967159e-07   
3413  0.000020     1    0   0   0        5  0.667527  0.667527  2.059852e-07   
3414  0.000022     1    0   0   0        5  0.667527  0.667527  2.286436e-07   
3415  0.000016     1    0   0   0        5  0.667527  0.667527  1.647882e-07   
3416  0.000015     1    0   0   0        5  0.667527  0.667527  1.524291e-07   

                    
              perr  
3289  3.219948e-08  
3290  4.154772e-08  
3291  4.154772e-08  
3292  3.219948e-08  
3293  2.908340e-08  
...            ...  
3412  1.983904e-07  
3413  2.077386e-07  
3414  2.305898e-07  
3415  1.661909e-07  
3416  1.537266e-07  

[128 rows x 16 columns]

time: 191 ms (started: 2023-02-07 10:48:15 -08:00)


In [29]:
file_id_view_top.agg({('trange', ''): ['unique', 'nunique'], ('file_id', ''): ['unique', 'nunique'], ('proc_id', ''): ['unique', 'nunique']})

,trange,file_id,proc_id
,,,
unique,"[4, 5, 6, 7]",[4531734247156679037],"[6860131432592370324, 6860131432594991764, 6860131432597613204, 6860131432600234644, 6860131432602856084, 6860131432605477524, 6860131432608098964, 6860131432610720404, 6860131432613341844, 6860131432615963284, 6860131432618584724, 6860131432621206164, 6860131432623827604, 6860131432626449044, 6860131432629070484, 6860131432631691924, 6860131432634313364, 6860131432636934804, 6860131432639556244, 6860131432642177684, 6860131432644799124, 6860131432647420564, 6860131432650042004, 6860131432652663444, 6860131432655284884, 6860131432657906324, 6860131432660527764, 6860131432663149204, 6860131432665770644, 6860131432668392084, 6860131432671013524, 6860131432589748884]"
nunique,4,1,32


time: 23.8 ms (started: 2023-02-07 10:48:37 -08:00)


In [17]:
views[('trange', 'proc_id', 'file_id')].compute().groupby(['file_id', 'io_cat']).agg({('duration', 'sum'): sum, ('duration', 'csp'): 'first', ('duration', 'delta'): 'first', ('duration', 'score'): 'first'}).sort_values(('duration', 'sum'), ascending=False).groupby([('duration', 'score'), 'io_cat']).count()

duration          
                              sum csp delta
(duration, score) io_cat                   
1                 2            27  27    27
                  3            29  29    29
2                 2             9   9     9
                  3             9   9     9
5                 2             1   1     1
                  3             1   1     1

time: 417 ms (started: 2023-02-07 10:17:47 -08:00)


In [16]:
file_id_view = views[('file_id',)].compute()
file_id_view[file_id_view[('duration', 'score')] == 5].groupby(['file_id', 'io_cat']).sum()

trange       proc_id acc_pat       duration  index  \
                                                    min max        sum  count   
file_id             io_cat                                                      
4531734247156679037 2        13.0  1.372026e+19       0   0   0.006853    3.0   
                    3       695.0  8.643766e+20       0   0  64.806252  159.0   

                                  size                         duration  \
                                   min         max         sum    score   
file_id             io_cat                                                
4531734247156679037 2       27787264.0  27787264.0  44564480.0     10.0   
                    3              0.0         0.0         0.0    630.0   

                                                                      
                                  csp      delta      pero      perr  
file_id             io_cat                                            
4531734247156679037 2        1.335054   1.335054  0.000071  0.000071  
                    3       84.108391  84.108391  0.667456  0.673138

time: 169 ms (started: 2023-02-07 10:15:02 -08:00)


In [10]:
views[('proc_id',)].compute().groupby('proc_id').agg({('duration', 'sum'): sum, ('duration', 'csp'): 'first', ('duration', 'delta'): 'first', ('duration', 'score'): 'first'}).sort_values(('duration', 'sum'), ascending=False).groupby([('duration', 'score')]).count()

duration          
                       sum csp delta
(duration, score)                   
2                       32  32    32

time: 168 ms (started: 2023-02-07 10:11:14 -08:00)


In [15]:
from wisio._recorder.analysis import delta_bins

bins = delta_bins(0.0001)
print(bins)
np.digitize([0.002, 2], bins)

[0, 0.0001, 0.001, 0.01, inf]


array([3, 4])

time: 5.87 ms (started: 2023-02-07 09:39:58 -08:00)


In [9]:
%autoreload 2

recorder_analyzer._save_views(log_dir='/p/vast1/iopp/wisio/computed_llcs/cm1/stage1', views=views)


time: 1.55 s (started: 2023-02-05 20:28:22 -08:00)


In [10]:
pd.read_parquet("/p/vast1/iopp/wisio/computed_llcs/cm1/stage1/trange/part.0.parquet")

,trange,file_id,proc_id,io_cat,acc_pat_min,acc_pat_max,duration_sum,index_count,size_min,size_max,size_sum
__null_dask_index__,,,,,,,,,,,
3,3,901974072163427748,6860131432589814420,3,0,0,0.000003,1,0,0,0
4,3,901974072163427748,6860131432589879956,3,0,0,0.000003,1,0,0,0
5,3,901974072163427748,6860131432589945492,3,0,0,0.000002,1,0,0,0
6,3,901974072163427748,6860131432590011028,3,0,0,0.000003,1,0,0,0
7,3,901974072163427748,6860131432590076564,3,0,0,0.000002,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
10257,669,4531734246508946226,6860131432660527764,3,0,0,0.000006,1,0,0,0
10258,669,4531734246508946226,6860131432663149204,3,0,0,0.000005,1,0,0,0
10259,669,4531734246508946226,6860131432665770644,3,0,0,0.000005,1,0,0,0


time: 73.5 ms (started: 2023-02-05 20:28:27 -08:00)


In [12]:
proc_ids = llcviews[('trange', 'file_id', 'proc_id')] \
    .compute() \
    .set_index(['trange', 'file_id', 'proc_id', 'io_cat']) \
    .sort_values(('duration', 'sum'), ascending=False) \
    .loc[(5, 4531734247156679037,)] \
    .reset_index()['proc_id'] \
    .unique() 

proc_ids


array([6860131432629070484, 6860131432615963284, 6860131432594991764,
       6860131432613341844, 6860131432626449044, 6860131432652663444,
       6860131432623827604, 6860131432636934804, 6860131432647420564,
       6860131432655284884, 6860131432602856084, 6860131432671013524,
       6860131432608098964, 6860131432597613204, 6860131432663149204,
       6860131432660527764, 6860131432621206164, 6860131432631691924,
       6860131432600234644, 6860131432644799124, 6860131432610720404,
       6860131432605477524, 6860131432665770644, 6860131432668392084,
       6860131432650042004, 6860131432642177684, 6860131432618584724,
       6860131432634313364, 6860131432639556244, 6860131432657906324,
       6860131432592370324])

time: 40.3 ms (started: 2023-01-26 08:25:48 -08:00)


In [20]:
proc_id_df0 = llcviews[('proc_id', 'proc_id', 'proc_id')].compute().reset_index().groupby(['proc_id', 'io_cat']).agg({('duration', 'sum'):[sum]}).sort_values(('duration', 'sum', 'sum'), ascending=False)
proc_id_df0[proc_id_df0[('duration', 'sum', 'sum')] > 3].reset_index()['proc_id'].unique()

array([6860131432589748884, 6860131432629070484, 6860131432615963284,
       6860131432636934804, 6860131432613341844, 6860131432626449044,
       6860131432671013524, 6860131432602856084, 6860131432594991764,
       6860131432623827604, 6860131432657906324, 6860131432655284884,
       6860131432644799124, 6860131432610720404])

time: 37.7 ms (started: 2023-01-26 08:36:54 -08:00)


In [38]:
def filter_delta(df: pd.DataFrame, delta: float, metric='duration'):
    df[metric, 'csp'] = df[metric, 'sum'].cumsum() / df[metric, 'sum'].sum()
    df[metric, 'delta'] = df[metric, 'csp'].diff().fillna(df[metric, 'csp'])
    df_filtered = df[df[metric, 'delta'] > delta]
    df_filtered = df_filtered.reindex(sorted(df_filtered.columns), axis=1)
    return df_filtered


time: 1.6 ms (started: 2023-01-26 08:53:37 -08:00)


In [32]:
proc_id_df = llcviews[('trange', 'file_id', 'proc_id')].compute().reset_index().groupby(['proc_id', 'io_cat']).agg({('duration', 'sum'):[sum]}).sort_values(('duration', 'sum', 'sum'), ascending=False)
bottleneck_proc_ids = proc_id_df[(proc_id_df[('duration', 'sum', 'sum')] > 3)].reset_index()['proc_id'].unique()
bottleneck_proc_ids

array([6860131432589748884, 6860131432629070484, 6860131432615963284,
       6860131432636934804, 6860131432613341844, 6860131432626449044,
       6860131432671013524, 6860131432602856084, 6860131432594991764,
       6860131432623827604, 6860131432657906324, 6860131432655284884,
       6860131432644799124, 6860131432610720404])

time: 45.1 ms (started: 2023-01-26 08:49:13 -08:00)


In [42]:
proc_id_df = llcviews[('trange', 'file_id', 'proc_id')].compute().reset_index()
# proc_id_df
proc_id_df2 = proc_id_df[(proc_id_df['proc_id'].isin(bottleneck_proc_ids)) & (proc_id_df['io_cat'] == 3)].sort_values(('duration', 'sum'), ascending=False)
filter_delta(df=proc_id_df2, delta=0.0001)['file_id'].unique()
# list(proc_id_df2[proc_id_df2['duration', 'sum'] > 2]['file_id'].unique())

array([4531734247156679037, 4531734245058167199, 4531734244909082846,
       4531734247515089012, 4531734246187791667, 4531734247572069952,
       4531734247569105940, 4531734247135877453, 4531734247394096880,
       4531734245572410724, 4531734247092377932, 4531734245256545024,
       4531734245629303532, 4531734247071939659, 4531734246726558288,
       4531734248478099672, 4531734247914523647, 4531734245001276449,
       4531734244977734007, 4531734246458134144, 4531734244818756329,
       4531734247415627620, 4531734247371220322, 4531734248556487453,
       4531734247452668476, 4531734245570931047, 4531734245212906884,
       4531734247416422946, 4531734248037210752, 4531734246877945517])

time: 34.5 ms (started: 2023-01-26 08:55:03 -08:00)


In [43]:
proc_id_df3 = llcviews[('file_id', 'trange', 'proc_id')].compute().reset_index().groupby(['proc_id', 'io_cat']).agg({('duration', 'sum'):[sum]}).sort_values(('duration', 'sum', 'sum'), ascending=False)
bottleneck_proc_ids3 = proc_id_df3[(proc_id_df3[('duration', 'sum', 'sum')] > 3)].reset_index()['proc_id'].unique()
bottleneck_proc_ids3

array([6860131432589748884, 6860131432629070484, 6860131432615963284,
       6860131432636934804, 6860131432613341844, 6860131432626449044,
       6860131432671013524, 6860131432602856084, 6860131432594991764,
       6860131432623827604, 6860131432657906324, 6860131432655284884,
       6860131432644799124, 6860131432610720404])

time: 45.5 ms (started: 2023-01-26 08:56:13 -08:00)


In [44]:
proc_id_df4 = llcviews[('file_id', 'trange', 'proc_id')].compute().reset_index()
# proc_id_df
proc_id_df4 = proc_id_df4[(proc_id_df4['proc_id'].isin(bottleneck_proc_ids3)) & (proc_id_df4['io_cat'] == 3)].sort_values(('duration', 'sum'), ascending=False)
filter_delta(df=proc_id_df4, delta=0.0001)['file_id'].unique()

array([4531734247156679037, 4531734245058167199, 4531734244909082846,
       4531734247515089012, 4531734246187791667, 4531734247572069952,
       4531734247569105940, 4531734247135877453, 4531734247394096880,
       4531734245572410724, 4531734247092377932, 4531734245256545024,
       4531734245629303532, 4531734247071939659, 4531734246726558288,
       4531734248478099672, 4531734247914523647, 4531734245001276449,
       4531734244977734007, 4531734246458134144, 4531734244818756329,
       4531734247415627620, 4531734247371220322, 4531734248556487453,
       4531734247452668476, 4531734245570931047, 4531734245212906884,
       4531734247416422946, 4531734248037210752, 4531734246877945517])

time: 34.6 ms (started: 2023-01-26 08:56:44 -08:00)


In [ ]:
# llcviews[('trange', 'file_id', 'proc_id')].compute().

In [31]:
llcviews[('trange', 'file_id', 'proc_id')] \
    .compute() \
    .set_index(['trange', 'file_id', 'proc_id', 'io_cat']) \
    .sort_values(('duration', 'sum'), ascending=False)


acc_pat      duration  \
                                                          min max       sum   
trange file_id             proc_id             io_cat                         
5      4531734247156679037 6860131432629070484 3            0   0  2.191843   
                           6860131432615963284 3            0   0  2.191390   
                           6860131432594991764 3            0   0  2.167557   
                           6860131432613341844 3            0   0  2.165936   
                           6860131432626449044 3            0   0  2.164769   
...                                                       ...  ..       ...   
3      901974072163427748  6860131432613341844 3            0   0  0.000002   
                           6860131432610720404 3            0   0  0.000002   
                           6860131432642177684 3            0   0  0.000002   
                           6860131432594991764 3            0   0  0.000002   
                           6860131432618584724 3            0   0  0.000002   

                                                      index size          
                                                      count  min max sum  
trange file_id             proc_id             io_cat                     
5      4531734247156679037 6860131432629070484 3          1    0   0   0  
                           6860131432615963284 3          1    0   0   0  
                           6860131432594991764 3          1    0   0   0  
                           6860131432613341844 3          1    0   0   0  
                           6860131432626449044 3          1    0   0   0  
...                                                     ...  ...  ..  ..  
3      901974072163427748  6860131432613341844 3          1    0   0   0  
                           6860131432610720404 3          1    0   0   0  
                           6860131432642177684 3          1    0   0   0  
                           6860131432594991764 3          1    0   0   0  
                           6860131432618584724 3          1    0   0   0  

[6518 rows x 7 columns]

time: 58.4 ms (started: 2023-01-26 08:46:31 -08:00)


In [111]:
df_1 = llcviews[('trange', 'trange', 'trange')].compute().sort_values(('duration', 'sum'), ascending=False)
df_2 = llcviews[('trange', 'trange', 'file_id')].compute().sort_values(('duration', 'sum'), ascending=False)
df_3 = llcviews[('trange', 'trange', 'proc_id')].compute().sort_values(('duration', 'sum'), ascending=False)


time: 47.7 ms (started: 2023-01-25 21:33:22 -08:00)


In [112]:
df_all = pd.concat([ddf.compute() for ddf in llcviews.values()])
df_all[['trange', 'file_id', 'proc_id', 'io_cat']].value_counts()


(trange, )  (file_id, )          (proc_id, )          (io_cat, )
669         4531734246508946226  6860131432671013524  3             27
345         4531734247135877453  6860131432671013524  3             27
            4531734247569105940  6860131432589748884  3             27
                                 6860131432594991764  3             27
                                 6860131432602856084  3             27
                                                                    ..
3           901974072163427748   6860131432671865492  3              9
1           4531734244992817467  6860131432590862996  3              5
4           4531734244992817467  6860131432603839124  1              5
                                                      3              5
1           4531734244992817467  6860131432590862996  1              5
Length: 10260, dtype: int64

time: 323 ms (started: 2023-01-25 21:33:24 -08:00)


In [45]:
llc_value_counts = dd.concat(list(llcviews.values())) \
    .groupby(['trange', 'file_id', 'proc_id', 'io_cat']) \
    .count()['index', 'count'] \
    .compute() \
    .sort_values(ascending=False) \
    .rename(('row', 'nrepeat'))

llc_value_counts


trange
3      48159
668     6785
345     4823
450     3896
562     3599
       ...  
395       90
41        90
277       81
237       72
1         10
Name: (row, nrepeat), Length: 393, dtype: int64

time: 611 ms (started: 2023-01-26 08:57:17 -08:00)


In [114]:
df_1.merge(llc_value_counts, on=['trange', 'file_id', 'proc_id', 'io_cat'], how='left')

trange              file_id              proc_id io_cat acc_pat      \
                                                                 min max   
0         5  4531734247156679037  6860131432629070484      3       0   0   
1         5  4531734247156679037  6860131432615963284      3       0   0   
2         5  4531734247156679037  6860131432594991764      3       0   0   
3         5  4531734247156679037  6860131432613341844      3       0   0   
4         5  4531734247156679037  6860131432626449044      3       0   0   
...     ...                  ...                  ...    ...     ...  ..   
6040      3   901974072163427748  6860131432607443604      3       0   0   
6041      3   901974072163427748  6860131432607509140      3       0   0   
6042      3   901974072163427748  6860131432607705748      3       0   0   
6043      3   901974072163427748  6860131432607902356      3       0   0   
6044      3   901974072163427748  6860131432664132244      3       0   0   

      duration index size             row  
           sum count  min max sum nrepeat  
0     2.191843     1    0   0   0      27  
1     2.191390     1    0   0   0      27  
2     2.167557     1    0   0   0      27  
3     2.165936     1    0   0   0      27  
4     2.164769     1    0   0   0      27  
...        ...   ...  ...  ..  ..     ...  
6040  0.000002     1    0   0   0       9  
6041  0.000002     1    0   0   0       9  
6042  0.000002     1    0   0   0       9  
6043  0.000002     1    0   0   0       9  
6044  0.000002     1    0   0   0       9  

[6045 rows x 12 columns]

time: 37.9 ms (started: 2023-01-25 21:33:27 -08:00)


In [115]:
df_2.merge(llc_value_counts, on=['trange', 'file_id', 'proc_id', 'io_cat'], how='left')

trange              file_id              proc_id io_cat acc_pat      \
                                                                 min max   
0         5  4531734247156679037  6860131432629070484      3       0   0   
1         5  4531734247156679037  6860131432615963284      3       0   0   
2         5  4531734247156679037  6860131432594991764      3       0   0   
3         5  4531734247156679037  6860131432613341844      3       0   0   
4         5  4531734247156679037  6860131432626449044      3       0   0   
...     ...                  ...                  ...    ...     ...  ..   
6101      4  4531734247156679037  6860131432621206164      3       0   0   
6102      4  4531734247156679037  6860131432602856084      3       0   0   
6103      6  4531734247156679037  6860131432602856084      3       0   0   
6104      4  4531734247156679037  6860131432655284884      3       0   0   
6105      6  4531734247156679037  6860131432615963284      3       0   0   

      duration index size             row  
           sum count  min max sum nrepeat  
0     2.191843     1    0   0   0      27  
1     2.191390     1    0   0   0      27  
2     2.167557     1    0   0   0      27  
3     2.165936     1    0   0   0      27  
4     2.164769     1    0   0   0      27  
...        ...   ...  ...  ..  ..     ...  
6101  0.000003     1    0   0   0      18  
6102  0.000003     1    0   0   0      18  
6103  0.000003     2    0   0   0      27  
6104  0.000003     1    0   0   0      18  
6105  0.000003     2    0   0   0      27  

[6106 rows x 12 columns]

time: 36.1 ms (started: 2023-01-25 21:33:28 -08:00)


In [116]:
df_3.merge(llc_value_counts, on=['trange', 'file_id', 'proc_id', 'io_cat'], how='left')

trange              file_id              proc_id io_cat acc_pat      \
                                                                 min max   
0         5  4531734247156679037  6860131432629070484      3       0   0   
1         5  4531734247156679037  6860131432615963284      3       0   0   
2         5  4531734247156679037  6860131432594991764      3       0   0   
3         5  4531734247156679037  6860131432613341844      3       0   0   
4         5  4531734247156679037  6860131432626449044      3       0   0   
...     ...                  ...                  ...    ...     ...  ..   
6513      3   901974072163427748  6860131432613341844      3       0   0   
6514      3   901974072163427748  6860131432610720404      3       0   0   
6515      3   901974072163427748  6860131432642177684      3       0   0   
6516      3   901974072163427748  6860131432594991764      3       0   0   
6517      3   901974072163427748  6860131432618584724      3       0   0   

      duration index size             row  
           sum count  min max sum nrepeat  
0     2.191843     1    0   0   0      27  
1     2.191390     1    0   0   0      27  
2     2.167557     1    0   0   0      27  
3     2.165936     1    0   0   0      27  
4     2.164769     1    0   0   0      27  
...        ...   ...  ...  ..  ..     ...  
6513  0.000002     1    0   0   0      18  
6514  0.000002     1    0   0   0      18  
6515  0.000002     1    0   0   0      18  
6516  0.000002     1    0   0   0      18  
6517  0.000002     1    0   0   0      18  

[6518 rows x 12 columns]

time: 36.4 ms (started: 2023-01-25 21:33:28 -08:00)


In [8]:
from wisio._recorder.analysis import format_df

last_llc_ddf_grp = last_llc_ddf.groupby(['trange']).sum().reset_index()
last_llc_ddf_grp['duration', 'gsum'] = last_llc_ddf_grp['duration', 'sum']
last_llc_ddf_grp = format_df(df=last_llc_ddf_grp, stats_df=stats_df)
last_llc_ddf_grp

acc_pat            bw                           duration            \
       max min       per           sum   sum_fmt        csp     delta   
0        0   0  0.000000  0.000000e+00  0.00GB/s   0.979886  0.000214   
1        0   0  0.000000  0.000000e+00  0.00GB/s  11.593538  0.688280   
2        0   0  0.122351  2.155673e+09  2.01GB/s   1.958009  0.000404   
3        0   0  0.000116  2.044043e+06  0.00GB/s   0.978528  0.000238   
4        0   0  0.000257  4.534855e+06  0.00GB/s  28.467251  0.038133   
5        0   0  0.002134  3.759181e+07  0.04GB/s  18.116020  0.023181   
6        0   0  0.000000  0.000000e+00  0.00GB/s   0.974251  0.000312   
7        0   0  0.000088  1.556256e+06  0.00GB/s   0.973619  0.000327   
8        0   0  0.000127  2.245698e+06  0.00GB/s   0.979447  0.000226   
9        0   0  0.000501  8.833168e+06  0.01GB/s  27.327360  0.039155   
10       0   0  0.463247  8.161868e+09  7.60GB/s   0.983250  0.000107   
11       0   0  0.000458  8.074231e+06  0.01GB/s  29.769077  0.042834   
12       0   0  0.000000  0.000000e+00  0.00GB/s   8.874919  0.011295   
13       0   0  0.000096  1.693292e+06  0.00GB/s   0.974856  0.000300   
14       0   0  0.000107  1.891343e+06  0.00GB/s   0.977530  0.000257   
15       0   0  0.000000  0.000000e+00  0.00GB/s   5.533835  0.005663   
16       0   0  0.000907  1.598175e+07  0.01GB/s  20.642870  0.054525   
17       0   0  0.000140  2.473208e+06  0.00GB/s   0.980092  0.000206   
18       0   0  0.000779  1.372067e+07  0.01GB/s  40.232132  0.025207   
19       0   0  0.407490  7.179497e+09  6.69GB/s   0.982224  0.000121   
20       0   0  0.000101  1.786610e+06  0.00GB/s   0.975958  0.000272   
21       0   0  0.000009  1.517495e+05  0.00GB/s   0.691631  0.003351   
22       0   0  0.000000  0.000000e+00  0.00GB/s  16.641649  0.003502   
23       0   0  0.001091  1.921714e+07  0.02GB/s  23.158850  0.045345   

                                    ...        size                     \
         gsum       per        sum  ...         max          mean  min   
0    0.020722  0.000213   0.020722  ...         0.0  0.000000e+00  0.0   
1   66.775764  0.687741  66.775764  ...         0.0  0.000000e+00  0.0   
2    0.039221  0.000404   0.039221  ...  16784086.0  1.207821e+07  0.0   
3    0.023064  0.000238   0.023064  ...     47144.0  7.857333e+03  0.0   
4    3.699615  0.038103   3.699615  ...  16777216.0  3.355443e+06  0.0   
5    2.248932  0.023162   2.248932  ...  16777216.0  8.454144e+06  0.0   
6    0.030239  0.000311   0.030239  ...         0.0  0.000000e+00  0.0   
7    0.031704  0.000327   0.031704  ...     49340.0  8.223333e+03  0.0   
8    0.021971  0.000226   0.021971  ...     49340.0  8.223333e+03  0.0   
9    3.798686  0.039124   3.798686  ...  16777216.0  5.592405e+06  0.0   
10   0.010358  0.000107   0.010358  ...  16777216.0  8.454144e+06  0.0   
11   4.155744  0.042801   4.155744  ...  16777216.0  5.592405e+06  0.0   
12   1.095861  0.011287   1.095861  ...         0.0  0.000000e+00  0.0   
13   0.029139  0.000300   0.029139  ...     49340.0  8.223333e+03  0.0   
14   0.024926  0.000257   0.024926  ...     47144.0  7.857333e+03  0.0   
15   0.549372  0.005658   0.549372  ...         0.0  0.000000e+00  0.0   
16   5.289874  0.054482   5.289874  ...  16777216.0  8.454144e+06  0.0   
17   0.019950  0.000205   0.019950  ...     49340.0  8.223333e+03  0.0   
18   2.445539  0.025187   2.445539  ...  16777216.0  5.592405e+06  0.0   
19   0.011775  0.000121   0.011775  ...  16777216.0  8.454144e+06  0.0   
20   0.026387  0.000272   0.026387  ...     47144.0  7.857333e+03  0.0   
21   0.325141  0.003349   0.325141  ...     49340.0  8.223333e+03  0.0   
22   0.339812  0.003500   0.339812  ...         0.0  0.000000e+00  0.0   
23   4.399273  0.045309   4.399273  ...  16777216.0  8.454144e+06  0.0   

                                 trange    xfer                   
         per         sum sum_fmt        max_fmt mean_fmt min_fmt  
0   0.000000         0.0  0.00GB      3    <4

time: 127 ms (started: 2023-01-12 22:18:18 -08:00)


In [10]:
import itertools as it
from pathlib import PurePath


def filename(file_id):
    path = unique_filenames[file_id]['filename']
    pure_path = PurePath(path)
    # print(pure_path.parent.name)
    if pure_path.parent.name:
        return f"../{pure_path.name}"
    return pure_path.name


def io_cat_fmt(io_cat_min, io_cat_max):
    return ''.join([['-', 'r', 'w', 'm'][io_cat] for io_cat in set([io_cat_min, io_cat_max])]).rjust(3, '-')


def procname(proc_id):
    app = unique_processes[proc_id]['app']
    hostname = unique_processes[proc_id]['hostname']
    rank = str(unique_processes[proc_id]['rank'])
    return '-'.join(filter(None, [app, hostname, rank]))


def acc_pat_fmt(acc_pat_min, acc_pat_max):
    return 'seq.' if all(x == 0 for x in [acc_pat_min, acc_pat_max]) else 'rand.'


bokeh_df = last_llc_ddf.merge(last_llc_ddf_grp, on=('trange',), how='left', suffixes=(None, '_y'))
bokeh_df = bokeh_df.sort_values(by=[('duration_y', 'gsum'), ('duration', 'sum')], ascending=[False, False])
# bokeh_df[('filename',)] = bokeh_df[("filename",)].apply(filename)
bokeh_df['trange_fmt'] = bokeh_df['trange'].apply(lambda x: f"{str(x-1).rjust(5, '0')}-{str(x).rjust(5, '0')}")
bokeh_df['filename'] = bokeh_df['file_id', ''].apply(filename)
bokeh_df['procname'] = bokeh_df['proc_id', ''].apply(procname)
bokeh_df['io_cat_fmt'] = bokeh_df.apply(lambda x: io_cat_fmt(x[('io_cat', 'min')], x[('io_cat', 'max')]), axis=1)
bokeh_df['acc_pat_fmt'] = bokeh_df.apply(lambda x: acc_pat_fmt(x[('acc_pat', 'min')], x[('acc_pat', 'max')]), axis=1)
bokeh_df


trange              file_id              proc_id acc_pat      \
                                                         max min   
0        5  4531734247156679037  6860131432629070484       0   0   
1        5  4531734247156679037  6860131432615963284       0   0   
2        5  4531734247156679037  6860131432594991764       0   0   
3        5  4531734247156679037  6860131432613341844       0   0   
4        5  4531734247156679037  6860131432626449044       0   0   
..     ...                  ...                  ...     ...  ..   
263    199  4531734247371220322  6860131432589748884       0   0   
265      3  4531734247452668476  6860131432589748884       0   0   
266    454  4531734245570931047  6860131432589748884       0   0   
281    562  4531734245256545024  6860131432589748884       0   0   
290    234  4531734246026030115  6860131432589748884       0   0   

               bw                          duration            ...  \
              per           sum   sum_fmt       csp     delta  ...   
0    0.000000e+00  0.000000e+00  0.00GB/s  0.022592  0.022592  ...   
1    0.000000e+00  0.000000e+00  0.00GB/s  0.045179  0.022587  ...   
2    0.000000e+00  0.000000e+00  0.00GB/s  0.067521  0.022342  ...   
3    0.000000e+00  0.000000e+00  0.00GB/s  0.089846  0.022325  ...   
4    0.000000e+00  0.000000e+00  0.00GB/s  0.112159  0.022313  ...   
..            ...           ...       ...       ...       ...  ...   
263  1.628321e-07  2.245698e+06  0.00GB/s  0.979447  0.000226  ...   
265  0.000000e+00  0.000000e+00  0.00GB/s  0.979886  0.000214  ...   
266  1.793285e-07  2.473208e+06  0.00GB/s  0.980092  0.000206  ...   
281  5.205744e-04  7.179497e+09  6.69GB/s  0.982224  0.000121  ...   
290  5.918046e-04  8.161868e+09  7.60GB/s  0.983250  0.000107  ...   

         size_y          xfer_y                    trange_fmt  \
            sum sum_fmt max_fmt mean_fmt min_fmt                
0           0.0  0.00GB    <4KB     <4KB    <4KB  00004-00005   
1           0.0  0.00GB    <4KB     <4KB    <4KB  00004-00005   
2           0.0  0.00GB    <4KB     <4KB    <4KB  00004-00005   
3           0.0  0.00GB    <4KB     <4KB    <4KB  00004-00005   
4           0.0  0.00GB    <4KB     <4KB    <4KB  00004-00005   
..          ...     ...     ...      ...     ...          ...   
263     49340.0  0.00GB   ~64KB    ~16KB    <4KB  00198-00199   
265         0.0  0.00GB    <4KB     <4KB    <4KB  00002-00003   
266     49340.0  0.00GB   ~64KB    ~16KB    <4KB  00453-00454   
281  84541440.0  0.08GB   ~16MB    ~16MB    <4KB  00561-00562   
290  84541440.0  0.08GB   ~16MB    ~16MB    <4KB  00233-00234   

                        filename procname io_cat_fmt acc_pat_fmt  
                                                                  
0         ../cm1out_000001_s.dat      600        --m        seq.  
1         ../cm1out_000001_s.dat      400        --m        seq.  
2         ../cm1out_000001_s.dat       80        --m        seq.  
3         ../cm1out_000001_s.dat      360        --m        seq.  
4         ../cm1out_000001_s.dat      560        --m        seq.  
..                           ...      ...        ...         ...  
263  ../cm1out_diag_000101_s.dat        0        -wm        seq.  
265                 ../proc.info        0        --m        seq.  
266  ../cm1out_diag_000243_s.dat        0        -wm        seq.  
281       ../cm1out_000006_w.dat        0        -wm        seq.  
290       ../cm1out_000003_w.dat        0        -wm        seq.  

[293 rows x 63 columns]

time: 180 ms (started: 2023-01-12 22:18:28 -08:00)


In [11]:
from bokeh.io import show, save, output_file
from bokeh.models import (ColumnDataSource, DataCube, GroupingInfo, NumberFormatter,
                          StringFormatter, SumAggregator, TableColumn, CustomJS)

# output_file(filename='test.html')

source = ColumnDataSource(bokeh_df.reset_index())

target = ColumnDataSource(data=dict(row_indices=[], labels=[]))

formatter = StringFormatter(font_style='bold')
duration_formatter = NumberFormatter(format='0.000%')

columns = [
    TableColumn(field='procname_', title='Time Range > File ID > Proc ID', width=80, formatter=formatter),
    TableColumn(field='duration_per', title='Duration %', width=40,  formatter=duration_formatter),
    TableColumn(field='io_cat_fmt_', title='I/O Type', width=20),
    TableColumn(field='acc_pat_fmt_', title='Acc. Pat.', width=20),
    TableColumn(field='size_sum_fmt', title='I/O Size', width=30),
    TableColumn(field='bw_sum_fmt', title='BW', width=30),
    TableColumn(field='xfer_min_fmt', title='Min. Xfer', width=30),
    TableColumn(field='xfer_max_fmt', title='Max. Xfer', width=30),
]

grouping = [
    GroupingInfo(getter='trange_fmt_', aggregators=[SumAggregator(field_='duration_per')]),
    GroupingInfo(getter='filename_', aggregators=[SumAggregator(field_='duration_per')]),
]

cube = DataCube(source=source, columns=columns, grouping=grouping, target=target, sizing_mode='stretch_width')

show(cube)


time: 152 ms (started: 2023-01-12 22:18:37 -08:00)


In [41]:
# import panel as pn
# pn.extension('tabulator')
# from bokeh.sampledata.autompg import autompg

# bokeh_table_df = bokeh_df.copy()
# # bokeh_table_df.rename(columns={('trange',''): 'trange'}, inplace=True)
# bokeh_table_df.columns = ['_'.join(col) for col in bokeh_table_df.columns.values]

# table = pn.widgets.Tabulator(bokeh_table_df, groupby=['trange_', 'file_id_'], height=600, configuration=dict(columnCalcs='both', dataTree=True))
# table

time: 1.31 ms (started: 2023-01-12 14:18:45 -08:00)


In [12]:
trange_agg_cut_piv = hlm_dfs['trange'].reset_index().pivot(index='trange', columns='io_cat', values=('duration', 'sum')).fillna(0).rename_axis(columns=None)
trange_agg_cut_piv.rename(columns={1:'read', 2: 'write', 3: 'metadata'}, inplace=True)
trange_agg_cut_piv

,read,write,metadata
trange,,,
3,0.036801,0.000000,0.063219
5,0.000000,0.000000,66.775764
6,0.000000,0.000000,0.018766
7,0.000000,0.017043,0.038968
60,0.000000,0.000000,0.024598
128,0.000000,0.000000,3.714100
129,0.000000,0.027116,2.242857
150,0.000000,0.000000,0.031797
185,0.000000,0.000000,0.033043


time: 38.5 ms (started: 2023-01-12 22:18:51 -08:00)


In [13]:
from bokeh.io import show
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.palettes import Category10_3

symbols = list(trange_agg_cut_piv.index)
impacts = list(trange_agg_cut_piv.columns)

source = ColumnDataSource(trange_agg_cut_piv)

stat_plot = figure(width=800, height=200, tools="xpan,xbox_zoom,xbox_select,reset", active_drag="xbox_select")
stat_plot.vbar_stack(impacts, width=0.7, x="trange", source=source, color=Category10_3, legend_label=impacts)

stat_plot.y_range.start = 0
stat_plot.xgrid.grid_line_color = None
stat_plot.legend.location = "top_right"
stat_plot.legend.click_policy="mute"

show(stat_plot)

time: 212 ms (started: 2023-01-12 22:18:52 -08:00)


In [17]:
import math
from bokeh.layouts import layout
from bokeh.plotting import figure, show
from bokeh.palettes import Category10
from bokeh.transform import cumsum
from bokeh.layouts import column, row
from bokeh.models.widgets import Tabs, Panel
from bokeh.models import Legend

data = stats_df.reset_index()
data['io_cat'] = ['read', 'write', 'metadata']
data['duration', 'angle'] = data['duration', 'per'] * 2*math.pi
data['duration', 'color'] = Category10[3]
data['duration', 'per_fmt'] = data['duration', 'per'].apply(lambda x: format(float(x) * 100, ".2f") + "%")
data['size', 'angle'] = data['size', 'per'] * 2*math.pi
data['size', 'color'] = Category10[3]
data['size', 'per_fmt'] = data['size', 'per'].apply(lambda x: format(float(x) * 100, ".2f") + "%")
data['index', 'angle'] = data['index', 'per'] * 2*math.pi
data['index', 'color'] = Category10[3]
data['index', 'per_fmt'] = data['index', 'per'].apply(lambda x: format(float(x) * 100, ".2f") + "%")

def pie_chart(by: str, title: str):
    p = figure(height=200, width=300, title=title, toolbar_location=None,
               tooltips=f"@io_cat_: @{by}_per_fmt")

    p.add_layout(Legend(location='center', orientation='horizontal', padding=3, spacing=6, glyph_width=14, glyph_height=14, label_height=6), 'below')

    p.wedge(x=0, y=0, radius=0.4,
            start_angle=cumsum(f'{by}_angle', include_zero=True), end_angle=cumsum(f'{by}_angle'),
            fill_color=f'{by}_color', line_color=None, legend_field='io_cat_', source=data)

    p.axis.axis_label = None
    p.axis.visible = False
    p.grid.grid_line_color = None
#     p.legend.location = "bottom_left"
#     p.legend.orientation = "horizontal"
#     p.legend.padding = 3
#     p.legend.spacing = 6
#     p.legend.glyph_width = 14
#     p.legend.glyph_height = 14
#     p.legend.label_height = 6

    return p


plot_dur = pie_chart(by='duration', title='Agg. Duration by I/O Category')
plot_ix = pie_chart(by='index', title="Total I/O Ops by I/O Category")
plot_size = pie_chart(by='size', title="Agg. Size by I/O Category")

panel_dur = Panel(child=column(stat_plot, cube), title='Time-based')
panel_ix = Panel(child=plot_ix, title='File-based')
panel_size = Panel(child=plot_size, title='Process-based')

tabs = Tabs(tabs=[panel_dur, panel_ix, panel_size])

show(row(column([plot_dur, plot_ix, plot_size]), column([tabs])))
# show(tabs)


time: 609 ms (started: 2023-01-12 22:21:01 -08:00)
